# QWOP RL Model Showcase

This notebook demonstrates the performance of our trained models:
1. **Stable Baselines3 PPO** - Baseline implementation
2. **Custom PPO** - Our implementation from scratch
3. **Custom PPO with Torso Penalty** - Custom reward shaping to penalize low torso height (poor posture)
4. **Custom PPO with Stride Reward** - Custom reward shaping to encourage coordinated limb movement

Run each section to watch the models play QWOP in your browser!

In [1]:
import gymnasium as gym
import qwop_gym  # This registers the QWOP-v1 environment
from stable_baselines3 import PPO
import time

browser_path = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
driver_path = "C:\\Program Files\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

## 1. Stable Baselines3 PPO

Evaluate the model trained using Stable Baselines3 library:

In [2]:
# Load the trained model
model_path = "Good Models/stable_baselines_ppo_1.zip"
print(f"Loading model from {model_path}...")
model = PPO.load(model_path)
print("Model loaded successfully!\n")

# Create environment for evaluation (with slower frameskip to watch better)
eval_env = gym.make(
    'QWOP-v1',
    browser=browser_path,
    driver=driver_path,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,  # Slower for better visualization
    max_episode_steps=3000,
    text_in_browser="🤖 Trained AI Playing"
)

print("Playing 5 episodes with the trained model...")
print("Watch the browser to see your AI in action!\n")

episode_rewards = []
episode_distances = []

for episode in range(5):
    obs, info = eval_env.reset()
    episode_reward = 0
    steps = 0
    
    print(f"Episode {episode + 1}:")
    
    while True:
        # Use the trained model to predict actions (deterministic for consistency)
        action, _states = model.predict(obs, deterministic=True)
        
        obs, reward, terminated, truncated, info = eval_env.step(action)
        episode_reward += reward
        steps += 1
        
        if terminated or truncated:
            distance = info.get('distance', 0)
            episode_rewards.append(episode_reward)
            episode_distances.append(distance)
            
            print(f"  Steps: {steps}")
            print(f"  Distance: {distance:.2f}m")
            print(f"  Total Reward: {episode_reward:.2f}")
            print(f"  {'✓ SUCCESS!' if info.get('is_success') else '✗ Failed'}\n")
            break
        
        time.sleep(0.02)  # Small delay to watch the game

print("\n" + "="*50)
print("PERFORMANCE SUMMARY")
print("="*50)
print(f"Average Distance: {sum(episode_distances)/len(episode_distances):.2f}m")
print(f"Best Distance: {max(episode_distances):.2f}m")
print(f"Average Reward: {sum(episode_rewards)/len(episode_rewards):.2f}")
print(f"Successes: {sum(1 for d in episode_distances if d >= 100)}/5")
print("="*50)

eval_env.close()
print("\n✓ Evaluation complete!")

Loading model from Good Models/stable_baselines_ppo_1.zip...
Model loaded successfully!

Playing 5 episodes with the trained model...
Watch the browser to see your AI in action!

Episode 1:
  Steps: 3000
  Distance: 72.57m
  Total Reward: -45.65
  ✗ Failed

Episode 2:
  Steps: 3000
  Distance: 71.07m
  Total Reward: -46.90
  ✗ Failed

Episode 3:
  Steps: 3000
  Distance: 72.57m
  Total Reward: -45.77
  ✗ Failed

Episode 4:
  Steps: 3000
  Distance: 71.07m
  Total Reward: -46.90
  ✗ Failed

Episode 5:
  Steps: 3000
  Distance: 72.57m
  Total Reward: -45.77
  ✗ Failed


PERFORMANCE SUMMARY
Average Distance: 71.97m
Best Distance: 72.57m
Average Reward: -46.20
Successes: 0/5

✓ Evaluation complete!


In [3]:
# Import necessary libraries and custom PPO implementation
import numpy as np
import torch
import torch.nn as nn
import gymnasium as gym
import qwop_gym

# Reload train_ppo module to pick up latest changes
import importlib
import train_ppo
importlib.reload(train_ppo)

# Import custom PPO components from train_ppo.py
from train_ppo import (
    ActorCritic,
    RolloutBuffer,
    create_qwop_env,
    train_ppo,
    evaluate_policy,
    save_checkpoint,
    load_checkpoint
)

print("✓ All modules imported successfully!")

# PPO Hyperparameters
TOTAL_STEPS = 50000
ROLLOUT_STEPS = 2048
BATCH_SIZE = 256
PPO_EPOCHS = 4
GAMMA = 0.995
GAE_LAMBDA = 0.95
CLIP_COEF = 0.2
LEARNING_RATE = 3e-4
VALUE_COEF = 0.5
ENTROPY_COEF = 0.01
MAX_GRAD_NORM = 0.5
HIDDEN_SIZE = 256

✓ All modules imported successfully!


## 2. Custom PPO Implementation

Evaluate our custom PPO agent trained from scratch:

In [4]:
# Load and evaluate the trained PPO agent
MODEL_PATH = "Good Models/custom_ppo.pt"

print(f"Loading model from: {MODEL_PATH}")

# Create evaluation environment
eval_env = create_qwop_env(
    browser_path=browser_path,
    driver_path=driver_path,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,
    max_episode_steps=5000,
    text_in_browser="🤖 PPO Agent Evaluation"
)

# Initialize policy and load checkpoint
state_dim = eval_env.observation_space.shape[0]
action_dim = eval_env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

eval_policy = ActorCritic(state_dim, action_dim, HIDDEN_SIZE).to(device)
checkpoint = load_checkpoint(MODEL_PATH, eval_policy, device)

print(f"✓ Model loaded (trained for {checkpoint['step']} steps)")
print("\nEvaluating agent for 5 episodes...")
print("Watch the browser to see your trained AI in action!\n")

# Evaluate the policy
results = evaluate_policy(
    env=eval_env,
    policy=eval_policy,
    device=device,
    num_episodes=5,
    verbose=True,
    render_delay=0.02
)

# Close environment
eval_env.close()
print("\n✓ Evaluation complete!")

Loading model from: Good Models/custom_ppo.pt
✓ Model loaded (trained for 500000 steps)

Evaluating agent for 5 episodes...
Watch the browser to see your trained AI in action!

Episode 1:
  Steps: 1249
  Distance: 100.54m
  Total Reward: 83.78
  ✓ SUCCESS!

Episode 2:
  Steps: 1295
  Distance: 100.77m
  Total Reward: 82.26
  ✓ SUCCESS!

Episode 3:
  Steps: 1284
  Distance: 101.38m
  Total Reward: 83.03
  ✓ SUCCESS!

Episode 4:
  Steps: 1249
  Distance: 100.58m
  Total Reward: 83.66
  ✓ SUCCESS!

Episode 5:
  Steps: 32
  Distance: 0.53m
  Total Reward: -10.82
  ✗ Failed


EVALUATION SUMMARY
Average Distance: 80.76m
Best Distance: 101.38m
Average Reward: 64.38
Success Rate: 80.0%

✓ Evaluation complete!


In [5]:
# Custom Reward Wrapper 1: Torso Penalty
class TorsoPenaltyWrapper(gym.Wrapper):
    """
    Wrapper to penalize low torso height (poor posture).
    Encourages the agent to maintain an upright running position.
    """
    
    def __init__(self, env, penalty_scale=0.1, y_threshold=1.5, verbose=False):
        super().__init__(env)
        self.prev_distance = 0
        self.steps_taken = 0
        self.torso_y_idx = 1  # Index for torso Y-level in observation
        self.y_threshold = y_threshold  # Minimum acceptable torso height
        self.penalty_scale = penalty_scale  # Scale for posture penalty
        self.verbose = verbose
        self.total_base_reward = 0
        self.total_penalty = 0
        self.episode_steps = 0
    
    def reset(self, **kwargs):
        if self.verbose and self.episode_steps > 0:
            print(f"\n[TorsoPenalty] Episode Summary:")
            print(f"  Total base reward: {self.total_base_reward:.2f}")
            print(f"  Total penalty: {self.total_penalty:.2f}")
            print(f"  Penalty scale: {self.penalty_scale}")
            print(f"  Average penalty per step: {self.total_penalty/self.episode_steps:.4f}")
        
        self.prev_distance = 0
        self.steps_taken = 0
        self.total_base_reward = 0
        self.total_penalty = 0
        self.episode_steps = 0
        return self.env.reset(**kwargs)
    
    def step(self, action):
        obs, base_reward, terminated, truncated, info = self.env.step(action)
        self.steps_taken += 1
        
        # Get Torso Y-Level
        y_torso = obs[self.torso_y_idx]

        posture_penalty = 0
        if y_torso > self.y_threshold:
            # Linear Penalty: -k * (y_threshold - y_torso)
            posture_penalty = -self.penalty_scale * (self.y_threshold - y_torso)

        custom_reward = base_reward + posture_penalty
        
        # Track cumulative rewards for debugging
        self.total_base_reward += base_reward
        self.total_penalty += posture_penalty
        
        # Verbose logging every 100 steps
        if self.verbose and self.steps_taken % 100 == 0:
            print(f"Step {self.steps_taken}: y_torso={y_torso:.3f}, "
                  f"base_reward={base_reward:.3f}, penalty={posture_penalty:.3f}, "
                  f"total_reward={custom_reward:.3f}")
        
        return obs, custom_reward, terminated, truncated, info

print("✓ TorsoPenaltyWrapper defined!")


# Custom Reward Wrapper 2: Stride Reward
class StrideRewardWrapper(gym.Wrapper):
    """
    Wrapper to encourage coordinated limb movement.
    Adds stride reward based on forward positioning of body parts.
    """
    
    def __init__(self, env, penalty_scale=0.5, y_threshold=1.5, verbose=False):
        super().__init__(env)
        self.prev_distance = 0
        self.steps_taken = 0
        self.head_x_idx = 5
        self.torso_x_idx = 0
        self.left_leg_x_idx = 20
        self.right_leg_x_idx = 50
        self.y_threshold = y_threshold
        self.penalty_scale = penalty_scale
        self.verbose = verbose
        self.total_base_reward = 0
        self.total_stride_reward = 0
        self.episode_steps = 0
    
    def reset(self, **kwargs):
        if self.verbose and self.episode_steps > 0:
            print(f"\n[StrideReward] Episode Summary:")
            print(f"  Total base reward: {self.total_base_reward:.2f}")
            print(f"  Total stride reward: {self.total_stride_reward:.2f}")
            print(f"  Penalty scale: {self.penalty_scale}")
            print(f"  Average stride reward per step: {self.total_stride_reward/self.episode_steps:.4f}")
        
        self.prev_distance = 0
        self.steps_taken = 0
        self.total_base_reward = 0
        self.total_stride_reward = 0
        self.episode_steps = 0
        return self.env.reset(**kwargs)
    
    def step(self, action):
        obs, base_reward, terminated, truncated, info = self.env.step(action)
        self.steps_taken += 1
        self.episode_steps += 1
        
        stride_reward = obs[self.head_x_idx]*obs[self.torso_x_idx]*obs[self.left_leg_x_idx]*obs[self.right_leg_x_idx]

        custom_reward = base_reward + stride_reward*self.penalty_scale
        
        # Track cumulative rewards for debugging
        self.total_base_reward += base_reward
        self.total_stride_reward += stride_reward*self.penalty_scale
        
        # Verbose logging every 100 steps
        if self.verbose and self.steps_taken % 100 == 0:
            print(f"Step {self.steps_taken}, "
                  f"base_reward={base_reward:.3f}, stride_reward={stride_reward*self.penalty_scale:.3f}, "
                  f"total_reward={custom_reward:.3f}")
        
        return obs, custom_reward, terminated, truncated, info

print("✓ StrideRewardWrapper defined!")

✓ TorsoPenaltyWrapper defined!
✓ StrideRewardWrapper defined!


## 3. Custom PPO with Torso Penalty

Evaluate the custom PPO agent trained with torso height penalty to discourage knee-scraping:

In [6]:
# Load and evaluate the custom PPO agent trained with torso penalty
MODEL_PATH = "Good Models/custom_ppo_and_torso_penalty.pt"
PENALTY_SCALE = 0.1  # Same as used during training
Y_THRESHOLD = 1.5

print(f"Loading model from: {MODEL_PATH}")

# Create evaluation environment
eval_env = create_qwop_env(
    browser_path=browser_path,
    driver_path=driver_path,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,
    max_episode_steps=5000,
    text_in_browser="🤖 Torso Penalty PPO Agent"
)

# Apply torso penalty wrapper (same as training)
eval_env = TorsoPenaltyWrapper(eval_env, penalty_scale=PENALTY_SCALE, y_threshold=Y_THRESHOLD, verbose=False)

# Initialize policy and load checkpoint
state_dim = eval_env.observation_space.shape[0]
action_dim = eval_env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

eval_policy = ActorCritic(state_dim, action_dim, HIDDEN_SIZE).to(device)
checkpoint = load_checkpoint(MODEL_PATH, eval_policy, device)

print(f"✓ Model loaded (trained for {checkpoint['step']} steps)")
print(f"✓ Torso penalty wrapper applied (penalizes low torso height)")
print("\nEvaluating agent for 5 episodes...")
print("Watch the browser to see your trained AI in action!\n")

# Evaluate the policy
results = evaluate_policy(
    env=eval_env,
    policy=eval_policy,
    device=device,
    num_episodes=5,
    verbose=True,
    render_delay=0.02
)

# Close environment
eval_env.close()
print("\n✓ Evaluation complete!")

Loading model from: Good Models/custom_ppo_and_torso_penalty.pt
✓ Model loaded (trained for 25000 steps)
✓ Torso penalty wrapper applied (penalizes low torso height)

Evaluating agent for 5 episodes...
Watch the browser to see your trained AI in action!

Episode 1:
  Steps: 996
  Distance: 20.50m
  Total Reward: -27.79
  ✗ Failed

Episode 2:
  Steps: 25
  Distance: 0.62m
  Total Reward: -10.54
  ✗ Failed

Episode 3:
  Steps: 33
  Distance: -0.52m
  Total Reward: -11.69
  ✗ Failed

Episode 4:
  Steps: 2484
  Distance: 44.75m
  Total Reward: -59.42
  ✗ Failed

Episode 5:
  Steps: 1461
  Distance: 27.27m
  Total Reward: -38.39
  ✗ Failed


EVALUATION SUMMARY
Average Distance: 18.53m
Best Distance: 44.75m
Average Reward: -29.57
Success Rate: 0.0%

✓ Evaluation complete!


## 4. Custom PPO with Stride Reward

Evaluate the custom PPO agent trained with stride-based reward shaping to encourage coordinated limb movement:

In [7]:
# Load and evaluate the custom PPO agent trained with stride reward
MODEL_PATH = "Good Models/custom_ppo_and_stride_reward.pt"
PENALTY_SCALE = 0.5  # Same as used during training
Y_THRESHOLD = 1.5

print(f"Loading model from: {MODEL_PATH}")

# Create evaluation environment
eval_env = create_qwop_env(
    browser_path=browser_path,
    driver_path=driver_path,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,
    max_episode_steps=5000,
    text_in_browser="🤖 Stride Reward PPO Agent"
)

# Apply stride reward wrapper (same as training)
eval_env = StrideRewardWrapper(eval_env, penalty_scale=PENALTY_SCALE, y_threshold=Y_THRESHOLD, verbose=False)

# Initialize policy and load checkpoint
state_dim = eval_env.observation_space.shape[0]
action_dim = eval_env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

eval_policy = ActorCritic(state_dim, action_dim, HIDDEN_SIZE).to(device)
checkpoint = load_checkpoint(MODEL_PATH, eval_policy, device)

print(f"✓ Model loaded (trained for {checkpoint['step']} steps)")
print(f"✓ Stride reward wrapper applied (encourages coordinated limb movement)")
print("\nEvaluating agent for 5 episodes...")
print("Watch the browser to see your trained AI in action!\n")

# Evaluate the policy
results = evaluate_policy(
    env=eval_env,
    policy=eval_policy,
    device=device,
    num_episodes=5,
    verbose=True,
    render_delay=0.02
)

# Close environment
eval_env.close()
print("\n✓ Evaluation complete!")

Loading model from: Good Models/custom_ppo_and_stride_reward.pt
✓ Model loaded (trained for 50000 steps)
✓ Stride reward wrapper applied (encourages coordinated limb movement)

Evaluating agent for 5 episodes...
Watch the browser to see your trained AI in action!

Episode 1:
  Steps: 2483
  Distance: 24.69m
  Total Reward: 418.84
  ✗ Failed

Episode 2:
  Steps: 2816
  Distance: 30.03m
  Total Reward: 445.70
  ✗ Failed

Episode 3:
  Steps: 2134
  Distance: 26.72m
  Total Reward: 374.36
  ✗ Failed

Episode 4:
  Steps: 1608
  Distance: 20.74m
  Total Reward: 391.97
  ✗ Failed

Episode 5:
  Steps: 1977
  Distance: 24.44m
  Total Reward: 377.39
  ✗ Failed


EVALUATION SUMMARY
Average Distance: 25.33m
Best Distance: 30.03m
Average Reward: 401.65
Success Rate: 0.0%

✓ Evaluation complete!
